In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import math
import json
from sklearn.model_selection import train_test_split, GridSearchCV
from datetime import datetime
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import wilcoxon, mannwhitneyu
from sklearn.ensemble import AdaBoostRegressor

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_tweets = pd.read_csv('/content/drive/My Drive/thesis/datasets/twitter comments/tweets_sentiment.csv')
df_news = pd.read_csv('/content/drive/My Drive/thesis/datasets/news articles/news_sentiment_updated2.csv')
df_political_news = pd.read_csv('/content/drive/My Drive/thesis/datasets/news articles/political_news_sentiment.csv')

In [ ]:
company_ticker = 'AAPL'
df_tweets.drop(columns='body', inplace=True)
df_tweets = df_tweets[df_tweets['ticker_symbol'] == company_ticker].reset_index(drop=True)
df_news = df_news[df_news['Company'] == 'Apple'].reset_index(drop=True)

In [ ]:
dates = df_tweets['date'].unique()
avg_sentiment = []
tweets_volume = []

for date in dates:
    temp = df_tweets[df_tweets['date'] == date]
    avg_sentiment.append(temp['sentiment_score'].mean())
    tweets_volume.append(len(temp))
  
df_tweet_scores = pd.DataFrame({'Date':dates, 'Avg_sentiment_tweets':avg_sentiment, '# tweets' : tweets_volume})

In [ ]:
df_news['Sentiment'] = df_news['Sentiment'].map({'positive': 1, 'neutral': 0, 'negative': -1})

dates = df_news['Date'].unique()
avg_sentiment = []
pos_count = []
neu_count = []
neg_count = []


for date in dates:
    temp = df_news[df_news['Date'] == date]
    avg_sentiment.append(temp['Sentiment'].mean())
    pos_count.append(len(temp[temp['Sentiment'] == 1]))
    neu_count.append(len(temp[temp['Sentiment'] == 0]))
    neg_count.append(len(temp[temp['Sentiment'] == -1]))

df_news_scores = pd.DataFrame({'Date':dates, 'Avg_sentiment_news':avg_sentiment, '# Positive news' : pos_count, 
                               '# Neutral news' : neu_count, '# Negative news' : neg_count})

In [ ]:
df_political_news['Sentiment'] = df_political_news['Sentiment'].map({'Positive': 1, 'Neutral': 0, 'Negative': -1})

dates = df_political_news['Date'].unique()
avg_sentiment = []
pos_count = []
neu_count = []
neg_count = []


for date in dates:
    temp = df_political_news[df_political_news['Date'] == date]
    avg_sentiment.append(temp['Sentiment'].mean())
    pos_count.append(len(temp[temp['Sentiment'] == 1]))
    neu_count.append(len(temp[temp['Sentiment'] == 0]))
    neg_count.append(len(temp[temp['Sentiment'] == -1]))

df_political_news_scores = pd.DataFrame({'Date':dates, 'Avg_sentiment_polit_news':avg_sentiment, '# Positive polit_news' : pos_count, 
                               '# Neutral polit_news' : neu_count, '# Negative polit_news' : neg_count})

In [ ]:
stock = yf.Ticker(company_ticker)

start = min(df_tweet_scores['Date'])
end = max(df_tweet_scores['Date'])

df = stock.history(start=pd.to_datetime(start).date(), end=pd.to_datetime(end).date())

In [ ]:
df = df.reset_index()[['Date', 'Close']]
df['Date'] = df['Date'].apply(lambda x: datetime.strftime(pd.to_datetime(x), '%Y-%m-%d'))

df = pd.merge(df,df_tweet_scores,on='Date',how='inner')
df = pd.merge(df,df_news_scores,on='Date',how='inner')
df = pd.merge(df,df_political_news_scores,on='Date',how='left')

In [ ]:
df.fillna(df.mean(), inplace=True)

<ipython-input-10-bf934754a5b1>:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.fillna(df.mean(), inplace=True)


In [ ]:
def train_test_data(feature_vector):
    data = df[feature_vector].copy()
    data['Close Previous'] = data['Close'].shift(1)

    for column in data.columns:
        if column != 'Close' and column != 'Close Previous':
            data[column] = data[column].shift(1)

    data.dropna(inplace=True)
    X = data.drop(columns='Close').copy()
    y = data[['Close']].copy()

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

    return X_train, X_test, y_train, y_test

In [ ]:
def create_model(X_train, y_train):
  model = AdaBoostRegressor()
  model.fit(X_train, y_train.values.ravel())
  return model

In [3]:
#Wilcoxon Signed Rank Test

def wilcoxon_signed_rank_test(arr1, arr2, metric):
  if metric != 'accuracy':
    statistic, p_value = wilcoxon(arr1, arr2, alternative='greater')
  else:
    statistic, p_value = wilcoxon(arr1, arr2, alternative='less')
  alpha = 0.05
  if p_value < alpha:
    return False #there is a difference
  return True #no difference

In [4]:
#Mann-Whitney U test

def mannwhitneyu_test(arr1, arr2, metric):
  if metric != 'accuracy':
    statistic, p_value = mannwhitneyu(arr1, arr2, alternative='greater')
  else:
    statistic, p_value = mannwhitneyu(arr1, arr2, alternative='less')
  alpha = 0.05
  if p_value < alpha:
    return False #there is a difference
  return True #no difference

In [ ]:
def make_predictions(X_test, y_test, model):
  y_test = y_test.to_numpy()
  y_pred = model.predict(X_test)

  test_rmse = math.sqrt(mean_squared_error(y_test,y_pred))
  test_mae = mean_absolute_error(y_test,y_pred) 

  y_pred_diff = [y_pred[i] - y_pred[i-1] for i in range(1, len(y_pred))]
  y_test_diff = [y_test[i] - y_test[i-1] for i in range(1, len(y_test))]

  y_pred_movement = [1 if diff > 0 else 0 for diff in y_pred_diff]
  y_test_movement = [1 if diff > 0 else 0 for diff in y_test_diff]

  accuracy = sum([1 for pred, actual in zip(y_pred_movement, y_test_movement) if pred == actual]) / len(y_test_movement)

  return test_rmse, test_mae, accuracy, y_pred

In [ ]:
model_results = {
    'Prices' : [['Close'], [], [], []],
    'Prices + Tweets' : [['Close', 'Avg_sentiment_tweets'], [], [], []],
    'Prices + News' : [['Close', 'Avg_sentiment_news', '# Positive news', '# Neutral news', '# Negative news'], [], [], []],
    'Prices + Political News' : [['Close', 'Avg_sentiment_polit_news', '# Positive polit_news', '# Neutral polit_news', '# Negative polit_news'], [], [], []],
    'Prices + Tweets + News' : [['Close', 'Avg_sentiment_tweets', 'Avg_sentiment_news', '# Positive news', '# Neutral news', '# Negative news'], [], [], []],
    'Prices + Tweets + Political News' : [['Close', 'Avg_sentiment_tweets', 'Avg_sentiment_polit_news', '# Positive polit_news', '# Neutral polit_news', '# Negative polit_news'], [], [], []],
    'Prices + News + Political News' : [['Close', 'Avg_sentiment_news', '# Positive news', '# Neutral news', '# Negative news', 'Avg_sentiment_polit_news', '# Positive polit_news', '# Neutral polit_news', '# Negative polit_news'], [], [], []],
    'Prices + Tweets + News + Political News' : [['Close', 'Avg_sentiment_tweets', 'Avg_sentiment_news', '# Positive news', '# Neutral news', '# Negative news', 'Avg_sentiment_polit_news', '# Positive polit_news', '# Neutral polit_news', '# Negative polit_news'], [], [], []]
}

In [5]:
test_results = {
    'RMSE Wilcoxon' : [],
    'MAE Wilcoxon' : [],
    'Accuracy Wilcoxon' : [],
    'RMSE Mann-Whitney' : [],
    'MAE Mann-Whitney' : [],
    'Accuracy Mann-Whitney' : []
}

In [ ]:
for i in range(10):
  for key, item in model_results.items():
    feature_vector = model_results[key][0]
    X_train, X_test, y_train, y_test = train_test_data(feature_vector)
    model = create_model(X_train, y_train)
    test_rmse, test_mae, accuracy, predicted_values = make_predictions(X_test, y_test, model)
    model_results[key][1].append(round(test_rmse, 2))
    model_results[key][2].append(round(test_mae, 2))
    model_results[key][3].append(round(accuracy, 2))

In [ ]:
with open('/content/drive/My Drive/thesis/notebooks/APPLE/model results/apple adaboost results.json', "w") as file:
    json.dump(model_results, file)

In [7]:
#test for RMSE values


baseline_rmse = model_results['Prices'][1]
for key, item in model_results.items():
  if key != 'Prices':
    wilcoxon_rmse_result = wilcoxon_signed_rank_test(baseline_rmse, item[1], 'rmse')

    if wilcoxon_rmse_result == False:
      print(f'There is a difference between model with Prices and model with {key} using wilcoxon\n')
    else:
      print(f'No difference between model with Prices and model with {key} using wilcoxon\n')

    mannwhitneyu_rmse_result = mannwhitneyu_test(baseline_rmse, item[1], 'rmse')
    
    if mannwhitneyu_rmse_result == False:
      print(f'There is a difference between model with Prices and model with {key} using mannwhitneyu\n')
    else:
      print(f'No difference between model with Prices and model with {key} using mannwhitneyu\n')

    test_results['RMSE Wilcoxon'].append(['Prices', key, wilcoxon_rmse_result])
    test_results['RMSE Mann-Whitney'].append(['Prices', key, mannwhitneyu_rmse_result])

No difference between model with Prices and model with Prices + Tweets using wilcoxon

There is a difference between model with Prices and model with Prices + Tweets using mannwhitneyu

There is a difference between model with Prices and model with Prices + News using wilcoxon

There is a difference between model with Prices and model with Prices + News using mannwhitneyu

No difference between model with Prices and model with Prices + Political News using wilcoxon

No difference between model with Prices and model with Prices + Political News using mannwhitneyu

There is a difference between model with Prices and model with Prices + Tweets + News using wilcoxon

There is a difference between model with Prices and model with Prices + Tweets + News using mannwhitneyu

No difference between model with Prices and model with Prices + Tweets + Political News using wilcoxon

No difference between model with Prices and model with Prices + Tweets + Political News using mannwhitneyu

No differe

/usr/local/lib/python3.10/dist-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/usr/local/lib/python3.10/dist-packages/scipy/stats/_morestats.py:3428: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


In [8]:
#test for MAE values


baseline_rmse = model_results['Prices'][2]
for key, item in model_results.items():
  if key != 'Prices':
    wilcoxon_rmse_result = wilcoxon_signed_rank_test(baseline_rmse, item[2], 'mae')

    if wilcoxon_rmse_result == False:
      print(f'There is a difference between model with Prices and model with {key} using wilcoxon\n')
    else:
      print(f'No difference between model with Prices and model with {key} using wilcoxon\n')

    mannwhitneyu_rmse_result = mannwhitneyu_test(baseline_rmse, item[2], 'mae')
    
    if mannwhitneyu_rmse_result == False:
      print(f'There is a difference between model with Prices and model with {key} using mannwhitneyu\n')
    else:
      print(f'No difference between model with Prices and model with {key} using mannwhitneyu\n')

    test_results['RMSE Wilcoxon'].append(['Prices', key, wilcoxon_rmse_result])
    test_results['RMSE Mann-Whitney'].append(['Prices', key, mannwhitneyu_rmse_result])

There is a difference between model with Prices and model with Prices + Tweets using wilcoxon

There is a difference between model with Prices and model with Prices + Tweets using mannwhitneyu

There is a difference between model with Prices and model with Prices + News using wilcoxon

There is a difference between model with Prices and model with Prices + News using mannwhitneyu

No difference between model with Prices and model with Prices + Political News using wilcoxon

There is a difference between model with Prices and model with Prices + Political News using mannwhitneyu

There is a difference between model with Prices and model with Prices + Tweets + News using wilcoxon

There is a difference between model with Prices and model with Prices + Tweets + News using mannwhitneyu

No difference between model with Prices and model with Prices + Tweets + Political News using wilcoxon

No difference between model with Prices and model with Prices + Tweets + Political News using mannwhit

In [9]:
#test for accuracy values


baseline_rmse = model_results['Prices'][3]
for key, item in model_results.items():
  if key != 'Prices':
    wilcoxon_rmse_result = wilcoxon_signed_rank_test(baseline_rmse, item[3], 'accuracy')

    if wilcoxon_rmse_result == False:
      print(f'There is a difference between model with Prices and model with {key} using wilcoxon\n')
    else:
      print(f'No difference between model with Prices and model with {key} using wilcoxon\n')

    mannwhitneyu_rmse_result = mannwhitneyu_test(baseline_rmse, item[3], 'accuracy')
    
    if mannwhitneyu_rmse_result == False:
      print(f'There is a difference between model with Prices and model with {key} using mannwhitneyu\n')
    else:
      print(f'No difference between model with Prices and model with {key} using mannwhitneyu\n')

    test_results['RMSE Wilcoxon'].append(['Prices', key, wilcoxon_rmse_result])
    test_results['RMSE Mann-Whitney'].append(['Prices', key, mannwhitneyu_rmse_result])

No difference between model with Prices and model with Prices + Tweets using wilcoxon

No difference between model with Prices and model with Prices + Tweets using mannwhitneyu

No difference between model with Prices and model with Prices + News using wilcoxon

No difference between model with Prices and model with Prices + News using mannwhitneyu

No difference between model with Prices and model with Prices + Political News using wilcoxon

No difference between model with Prices and model with Prices + Political News using mannwhitneyu

No difference between model with Prices and model with Prices + Tweets + News using wilcoxon

No difference between model with Prices and model with Prices + Tweets + News using mannwhitneyu

No difference between model with Prices and model with Prices + Tweets + Political News using wilcoxon

No difference between model with Prices and model with Prices + Tweets + Political News using mannwhitneyu

No difference between model with Prices and model 

In [10]:
with open('/content/drive/My Drive/thesis/notebooks/APPLE/model results/apple adaboost test results.json', "w") as file:
    json.dump(test_results, file)